In [1]:
!pip3 install --user numpy pandas scipy matplotlib
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
# import plotly.plotly as py
# import plotly.figure_factory as ff
# from plotly.graph_objs import *

In [2]:
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)
plotly.__version__

'3.1.0'

In [3]:
import math
import os
import shutil
import time

Use `plotly.offline.iplot()` when working offline in a Jupyter Notebook to display the plot in the notebook.

In [5]:
output = pd.read_csv('output/output.csv', sep=';')
output = output.query('MetricName == "Time"')
output = output.apply(pd.to_numeric, errors='ignore')
output.loc[:,'value'] = (output.MetricValue / 10**6)
output.head()

,Tool,View,ChangeSet,RunIndex,Iteration,PhaseName,MetricName,MetricValue,value
0,jastadd-ttc18-relast-emf-inc,Q1,1,0,0,Initialization,Time,183510119,183.510119
2,jastadd-ttc18-relast-emf-inc,Q1,1,0,0,Load,Time,285321797,285.321797
4,jastadd-ttc18-relast-emf-inc,Q2,1,0,0,Initialization,Time,176043764,176.043764
6,jastadd-ttc18-relast-emf-inc,Q2,1,0,0,Load,Time,246596896,246.596896
8,jastadd-ttc18-relast-emf-inc,Q1,2,0,0,Initialization,Time,179378789,179.378789


In [19]:
# compute min and max values, rounded to nearest log10-value
minY = math.floor(math.log10(output.value.min()))
maxY = math.log10(output.value.max())
print(minY, maxY)

-2 4.525140545608281


In [23]:
phase_names = dict(Load='01-Load', Initial='02-Initial', Update='03-Update')

good_colors = ['#000000','#e69f00','#56b4e9','#009e73','#f0e442','#0072b2','#d55e00','#cc79a7']
good_shapes = ['circle', 'square', 'triangle-down', 'cross']
good_dashes = ['dot', 'solid', 'dash', 'dashdot', 'longdash']
# color, shape, dash-type, name in legend
diagram_info = {
    'xml-inc':          (good_colors[0], good_shapes[0], good_dashes[0], "Incremental"),
    'xml-flush':        (good_colors[1], good_shapes[1], good_dashes[1], "Flushing"),
    'relast-xml-inc':   (good_colors[2], good_shapes[2], good_dashes[2], "Incremental (Relations)"),
    'relast-xml-flush': (good_colors[3], good_shapes[3], good_dashes[3], "Flushing (Relations)"),
    'relast-emf-inc':   (good_colors[4], good_shapes[0], good_dashes[0], "Incremental (EMF)"),
    'relast-emf-flush': (good_colors[7], good_shapes[1], good_dashes[1], "Flushing (EMF)")
}

In [8]:
def move_files(files, target_dir='diagrams/'):
    for source_path in files:
        if not source_path:
            continue
        while not os.path.exists(source_path):
            time.sleep(1)
        target_path = os.path.join(target_dir, os.path.basename(source_path))
        if os.path.exists(target_path):
            os.remove(target_path)
        shutil.move(source_path, target_path)

In [17]:
def axis_no(axes, no):
    return {} if axes is None or len(axes) <= no or axes[no] is None else axes[no]
def draw_scatter(df, query='Q1', phase='Load', download_file=True, axes=None, draw_legend=False):
    xaxis  = axis_no(axes, 0)
    yaxis  = axis_no(axes, 1)
    yaxis2 = axis_no(axes, 2)

    # create multiple scatters, filter df by Tool for each tool
    traces = []
    if 'mean' not in df.columns:
        mean_col = df.groupby(['Tool', 'ChangeSet'])['value'].mean() # don't reset the index!
        df = df.set_index(['Tool', 'ChangeSet']) # make the same index here
        df['mean'] = mean_col
        df = df.reset_index() # to take the hierarchical index off again
    for i, toolname in enumerate(df['Tool'].unique()):
        sub = df[df['Tool'] == toolname]
        name = toolname.replace('jastadd-ttc18-', '')
        color, shape, dash, legend_name = diagram_info[name]
        trace = go.Scatter(x = sub['ChangeSet'], y = sub['mean'],
                           name=legend_name,
                           mode = 'lines+markers',
                           line=dict(color=color, dash=dash),
                           marker=dict(symbol=shape, size=10, color=color))
        traces.append(trace)
    layout = go.Layout(#title='LiveContest Query {} {}'.format(query[1:], phase),
                       #plot_bgcolor='rgb(230, 230,230)',
                       #width=1600, height=1200,
                       xaxis=xaxis, yaxis=yaxis, yaxis2=yaxis2,
                       showlegend=draw_legend, legend=dict(orientation="h", x=-.1, y=1.2, font=dict(size=20)))
    fig = go.Figure(data=traces, layout=layout)

    if download_file:
        extension = 'png'
        source_filename = 'plot-{}-{}'.format(query, phase_names[phase])
        if draw_legend:
            source_filename += '-legend'
        source_path = os.path.join(os.path.expanduser('~/Downloads'), source_filename + '.' + extension)
        # remove the file beforehand to be able to check for its (new) existence afterwards
        if os.path.exists(source_path):
            os.remove(source_path)
        plotly.offline.iplot(fig, filename=source_filename, image=extension)
        return source_path
    else:
        plotly.offline.iplot(fig)

In [43]:
def create_axes(df, maxY=3, noyaxis=False):
    # craft a yaxis (execution time)
    tval = [minor * 10 ** major for major in range(minY, math.ceil(maxY)) for minor in range(1, 10)][:-5]
    ttxt = [""] * len(tval) if noyaxis else [(str(v) + " ") if i % 9 == 0 else "" for i,v in enumerate(tval)]
    yaxis=dict(type='log', range=[-0.05, maxY], tickvals=tval, ticktext=ttxt,
               title=None if noyaxis else 'Time (ms)', exponentformat='power',
               linecolor='black', mirror=True,
               gridcolor='#bdbdbd',
               tickfont=dict(size=28), titlefont=dict(size=32), automargin=True)
    
    # craft a xaxis (changeset)
    minX = math.log10(df.ChangeSet.min()) - 0.1
    maxX = math.log10(df.ChangeSet.max()) + 0.1
    xaxis = dict(type='log', title='ChangeSet', tickvals=[2**i for i in range(df.ChangeSet.max())], range=[minX, maxX],
                 linecolor='black', mirror=True,
                 gridcolor='#bdbdbd',
                 tickfont=dict(size=28), titlefont=dict(size=32))
    return (xaxis, yaxis)

In [45]:
printAll = True
saveFiles = True
files = []
first = True
for query in ('Q1', 'Q2') if printAll else ('Q1',):
    for phase in ('Load', 'Initial', 'Update') if printAll else ('Update',):
        new_data = output.query('View == @query & PhaseName == @phase')
        draw_it = lambda draw_legend: draw_scatter(new_data, query=query, phase=phase, download_file=saveFiles,
                                                   axes=create_axes(new_data, maxY=maxY),
                                                   draw_legend=draw_legend)
        if first:
            files.append(draw_it(True))
            first = False
        files.append(draw_it(False))
move_files(files)